In [1]:
param_len = 4145

list_url = ['https://www.centricparts.com/partFinder/page/index/?p=' + str(_+1) + '&product_list_limit=25&product_list_order=part_number' for _ in range(param_len)]

print('总数量：' + str(len(list_url)))
print()

from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_part = html.xpath('//li[@class="item product product-item"]')
                
                # = = = = = = = = = = = = = = =
                
                list_src = [list_text[0].strip() for list_text in [part.xpath('./div[@class="product-item-info"]/div[@class="img-container"]/img/@src') for part in list_part]]

                # = = = = = = = = = = = = = = =
                
                list_sku = [list_text[0].strip() for list_text in [part.xpath('./div[@class="product-item-info"]/div[@class="product details product-item-details"]/div[@class="sku-container"]/p[@class="sku-title"]/text()') for part in list_part]]
                
                # = = = = = = = = = = = = = = =
                
                list_url = [list_text[0].strip() for list_text in [part.xpath('./div[@class="product-item-info"]/div[@class="product details product-item-details"]/div[@class="product-name-container"]/strong[@class="product name product-item-name"]/a[@class="product-item-link"]/@href') for part in list_part]]
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'status': 'ok',
                                        'Menu': url,
                                        'Url': list_url,
                                        'Sku': list_sku,
                                        'Src': list_src})
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Menu': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./menu_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：4145

https://www.centricparts.com/partFinder/page/index/?p=1&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：4125 - 当前时间：13:50:05
https://www.centricparts.com/partFinder/page/index/?p=15&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：4124 - 当前时间：13:50:06
https://www.centricparts.com/partFinder/page/index/?p=8&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：4123 - 当前时间：13:50:07
https://www.centricparts.com/partFinder/page/index/?p=6&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：4122 - 当前时间：13:50:08
https://www.centricparts.com/partFinder/page/index/?p=20&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：4121 - 当前时间：13:50:09
https://www.centricparts.com/partFinder/page/index/?p=9&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：4120 - 当前时间：13:50:09
https://www.centricparts.com/partFinder/page/index/?p=18&product_list_limit=25&product_list_order=pa

https://www.centricparts.com/partFinder/page/index/?p=60&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：4069 - 当前时间：13:50:25
https://www.centricparts.com/partFinder/page/index/?p=56&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：4068 - 当前时间：13:50:26
https://www.centricparts.com/partFinder/page/index/?p=66&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：4067 - 当前时间：13:50:27
https://www.centricparts.com/partFinder/page/index/?p=64&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：4066 - 当前时间：13:50:27
https://www.centricparts.com/partFinder/page/index/?p=67&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：4065 - 当前时间：13:50:27
https://www.centricparts.com/partFinder/page/index/?p=63&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：4064 - 当前时间：13:50:27
https://www.centricparts.com/partFinder/page/index/?p=69&product_list_limit=25&product_list_order=part_num

https://www.centricparts.com/partFinder/page/index/?p=116&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：4013 - 当前时间：13:50:52
https://www.centricparts.com/partFinder/page/index/?p=71&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：4012 - 当前时间：13:50:52
https://www.centricparts.com/partFinder/page/index/?p=124&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：4011 - 当前时间：13:50:53
https://www.centricparts.com/partFinder/page/index/?p=126&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：4010 - 当前时间：13:50:54
https://www.centricparts.com/partFinder/page/index/?p=128&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：4009 - 当前时间：13:50:55
https://www.centricparts.com/partFinder/page/index/?p=122&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：4008 - 当前时间：13:50:55
https://www.centricparts.com/partFinder/page/index/?p=129&product_list_limit=25&product_list_order=pa

https://www.centricparts.com/partFinder/page/index/?p=175&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：3956 - 当前时间：13:51:14
https://www.centricparts.com/partFinder/page/index/?p=176&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：3955 - 当前时间：13:51:15
https://www.centricparts.com/partFinder/page/index/?p=177&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：3954 - 当前时间：13:51:15
https://www.centricparts.com/partFinder/page/index/?p=164&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：3953 - 当前时间：13:51:15
https://www.centricparts.com/partFinder/page/index/?p=178&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：3952 - 当前时间：13:51:15
https://www.centricparts.com/partFinder/page/index/?p=180&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：3951 - 当前时间：13:51:15
https://www.centricparts.com/partFinder/page/index/?p=181&product_list_limit=25&product_list_order=p

https://www.centricparts.com/partFinder/page/index/?p=234&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：3901 - 当前时间：13:51:36
https://www.centricparts.com/partFinder/page/index/?p=232&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：3900 - 当前时间：13:51:36
https://www.centricparts.com/partFinder/page/index/?p=233&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：3899 - 当前时间：13:51:37
https://www.centricparts.com/partFinder/page/index/?p=235&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：3898 - 当前时间：13:51:37
https://www.centricparts.com/partFinder/page/index/?p=236&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：3897 - 当前时间：13:51:38
https://www.centricparts.com/partFinder/page/index/?p=240&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：3896 - 当前时间：13:51:40
https://www.centricparts.com/partFinder/page/index/?p=239&product_list_limit=25&product_list_order=p

...

https://www.centricparts.com/partFinder/page/index/?p=3881&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：251 - 当前时间：14:24:06
https://www.centricparts.com/partFinder/page/index/?p=3884&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：250 - 当前时间：14:24:07
https://www.centricparts.com/partFinder/page/index/?p=3883&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：249 - 当前时间：14:24:07
https://www.centricparts.com/partFinder/page/index/?p=3880&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：248 - 当前时间：14:24:07
https://www.centricparts.com/partFinder/page/index/?p=3885&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：247 - 当前时间：14:24:07
https://www.centricparts.com/partFinder/page/index/?p=3886&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：246 - 当前时间：14:24:08
https://www.centricparts.com/partFinder/page/index/?p=3891&product_list_limit=25&product_list_order=

https://www.centricparts.com/partFinder/page/index/?p=3935&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：196 - 当前时间：14:24:37
https://www.centricparts.com/partFinder/page/index/?p=3936&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：195 - 当前时间：14:24:38
https://www.centricparts.com/partFinder/page/index/?p=3937&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：194 - 当前时间：14:24:38
https://www.centricparts.com/partFinder/page/index/?p=3939&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：193 - 当前时间：14:24:38
https://www.centricparts.com/partFinder/page/index/?p=3940&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：192 - 当前时间：14:24:39
https://www.centricparts.com/partFinder/page/index/?p=3938&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：191 - 当前时间：14:24:39
https://www.centricparts.com/partFinder/page/index/?p=3943&product_list_limit=25&product_list_order=

https://www.centricparts.com/partFinder/page/index/?p=3994&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：141 - 当前时间：14:25:06
https://www.centricparts.com/partFinder/page/index/?p=3997&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：140 - 当前时间：14:25:07
https://www.centricparts.com/partFinder/page/index/?p=3995&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：139 - 当前时间：14:25:07
https://www.centricparts.com/partFinder/page/index/?p=3998&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：138 - 当前时间：14:25:08
https://www.centricparts.com/partFinder/page/index/?p=4002&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：137 - 当前时间：14:25:09
https://www.centricparts.com/partFinder/page/index/?p=4005&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：136 - 当前时间：14:25:10
https://www.centricparts.com/partFinder/page/index/?p=4000&product_list_limit=25&product_list_order=

https://www.centricparts.com/partFinder/page/index/?p=4051&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：85 - 当前时间：14:25:37
https://www.centricparts.com/partFinder/page/index/?p=4052&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：84 - 当前时间：14:25:37
https://www.centricparts.com/partFinder/page/index/?p=3996&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：83 - 当前时间：14:25:38
https://www.centricparts.com/partFinder/page/index/?p=4054&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：82 - 当前时间：14:25:39
https://www.centricparts.com/partFinder/page/index/?p=4055&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：81 - 当前时间：14:25:39
https://www.centricparts.com/partFinder/page/index/?p=4050&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：80 - 当前时间：14:25:40
https://www.centricparts.com/partFinder/page/index/?p=4056&product_list_limit=25&product_list_order=part_n

https://www.centricparts.com/partFinder/page/index/?p=4106&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：30 - 当前时间：14:26:06
https://www.centricparts.com/partFinder/page/index/?p=4099&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：29 - 当前时间：14:26:07
https://www.centricparts.com/partFinder/page/index/?p=4115&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：28 - 当前时间：14:26:10
https://www.centricparts.com/partFinder/page/index/?p=4114&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：27 - 当前时间：14:26:11
https://www.centricparts.com/partFinder/page/index/?p=4110&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：26 - 当前时间：14:26:11
https://www.centricparts.com/partFinder/page/index/?p=4113&product_list_limit=25&product_list_order=part_number  <->  [ok] - 剩余数量：25 - 当前时间：14:26:11
https://www.centricparts.com/partFinder/page/index/?p=4117&product_list_limit=25&product_list_order=part_n